In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

import numpy as np
import math
import re
import time
from tqdm import tqdm

import pandas as pd
from bs4 import BeautifulSoup

from datasets import load_dataset

2023-01-10 17:30:15.796978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 17:30:15.946893: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-10 17:30:16.523396: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-10 17:30:16.523441: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Loading our data, spanish and english corpus and the non breaking prefixes

In [2]:
with open("/home/isma/Proyectos/Text_processing/Traductor/Files/P85-Non-Breaking-Prefix.en",
        mode= "r", encoding="utf-8") as f:
    nonBreaking_en = f.read()

with open("/home/isma/Proyectos/Text_processing/Traductor/Files/nonbreaking_prefix.es",
        mode= "r", encoding="utf-8") as f:
    nonBreaking_es = f.read()

nonBreaking_en = [' ' + pref + '.' for pref in nonBreaking_en.split("\n")]
nonBreaking_es = [' ' + pref + '.' for pref in nonBreaking_es.split("\n")]

nonBreaking_en[-10:-5], nonBreaking_es[-10:-5]

([' ms.', ' ph.', ' prof.', ' sr.', ' st.'],
 [' sra.', ' dr.', ' prof.', ' ing.', ' st.'])

In [3]:
new_dataset = load_dataset("un_pc", "en-es", cache_dir="./NewDataset")

Found cached dataset un_pc (/home/isma/Proyectos/Text_processing/Traductor/BiggerData/./NewDataset/un_pc/en-es/1.0.0/1360070a820db42f7427f5a98416dd3a1c956ae069b994bf2ec0b83ae16dcaee)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
train_dataset = new_dataset.get("train")[0:4500000]
train_dataset = train_dataset.get("translation")

In [5]:
element_en = train_dataset[30].get("en")
element_es = train_dataset[30].get("es")

In [6]:
import concurrent.futures

buffer = 4500
maxlen = 22 

def process_chunk(chunk, buffer, index):
    count = 0
    
    list_en = []
    list_es = []

    micro_list1 = []    
    micro_list2 = []

    for idx, element in enumerate(chunk):
        # Una vez que hemos llenado las pequeñas listas, pasamos a hacer una limpieza de todos los datos antes de añadirlo a nuestro conjunto más grande
        if len(micro_list1) > buffer-1:
            # Data cleaning
            for prefix in nonBreaking_en:
                for element1, element2 in zip(micro_list1, micro_list2):
                    element1 = element1.replace(prefix, prefix + '$$$')
                    element2 = element2.replace(prefix, prefix + '$$$')

                    element1 = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", element1)
                    element2 = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", element2)

                    element1 = re.sub(r"\.\$\$\$", '', element1)
                    element2 = re.sub(r"\.\$\$\$", '', element2)

                    element1 = re.sub(r"  +", " ", element1)
                    element2 = re.sub(r"  +", " ", element2)

            # Incorporate micro sections to full list, and reset microsections
            list_en = list_en + micro_list1
            micro_list1 = []
            
            list_es = list_es + micro_list2
            micro_list2 = []

            # Print informativo para saber la progresión
            if index == 1:
                count += buffer
                print("Elementos procesados: {}".format(count*15) )
                print("Tamaño del conjunto aproximado: {}\n".format(len(list_en*15)) ) 

        element1 = element.get("en")                         
        element2 = element.get("es") 

        if len(element1.split(" ")) & len(element2.split(" ")) <= maxlen:
            micro_list1.append(element.get("en"))
            micro_list2.append(element.get("es"))

    return [list_en, list_es]
        

chunks = []
chunk_size = int(len(train_dataset) / 15)
print("chunk_size: {}\n".format(chunk_size))

for i in range(0, len(train_dataset), chunk_size):
    chunks.append(train_dataset[i: i+chunk_size])

with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
    futures = [executor.submit(process_chunk, chunk, buffer, idx) for idx, chunk in enumerate(chunks)]
    concurrent.futures.wait(futures)

chunk_size: 300000

Elementos procesados: 67500
Tamaño del conjunto aproximado: 67500

Elementos procesados: 135000
Tamaño del conjunto aproximado: 135000

Elementos procesados: 202500
Tamaño del conjunto aproximado: 202500

Elementos procesados: 270000
Tamaño del conjunto aproximado: 270000

Elementos procesados: 337500
Tamaño del conjunto aproximado: 337500

Elementos procesados: 405000
Tamaño del conjunto aproximado: 405000

Elementos procesados: 472500
Tamaño del conjunto aproximado: 472500

Elementos procesados: 540000
Tamaño del conjunto aproximado: 540000

Elementos procesados: 607500
Tamaño del conjunto aproximado: 607500

Elementos procesados: 675000
Tamaño del conjunto aproximado: 675000

Elementos procesados: 742500
Tamaño del conjunto aproximado: 742500

Elementos procesados: 810000
Tamaño del conjunto aproximado: 810000

Elementos procesados: 877500
Tamaño del conjunto aproximado: 877500

Elementos procesados: 945000
Tamaño del conjunto aproximado: 945000

Elementos proces

In [7]:
final_list_es = []
final_list_en = []

for future in futures:
    final_list_en = final_list_en + future.result()[0]
    final_list_es = final_list_es + future.result()[1]

final_list_en[5:10], final_list_es[5:10], len(final_list_es)

(['THIRD MEETING OF THE PARTIES TO THE',
  'MONTREAL PROTOCOL ON SUBSTANCES',
  'THAT DEPLETE THE OZONE LAYER',
  'Nairobi, 1921 June 1991',
  'REPORT OF THE THIRD MEETING OF THE PARTIES'],
 ['TERCERA REUNION DE LAS PARTES EN EL PROTOCOLO',
  'DE MONTREAL RELATIVO A LAS SUSTANCIAS',
  'QUE AGOTAN LA CAPA DE OZONO',
  'Nairobi, 19 a 21 de junio de 1991',
  'INFORME DE LA TERCERA REUNION DE LAS PARTES EN EL'],
 3078000)

In [8]:
corpus_en = "\n".join(final_list_en)
corpus_es = "\n".join(final_list_es)

# Añadimos $$$ (algo que no encontraremos en el corpues) después de los puntos de frases sin fin
for prefix in tqdm(nonBreaking_en):
    corpus_en = corpus_en.replace(prefix, prefix + '$$$')
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)

# Eliminamos los marcadores $$$
corpus_en = re.sub(r"\.\$\$\$", '', corpus_en)

# Eliminamos espacios múltiples
corpus_en = re.sub(r"  +", " ", corpus_en)
corpus_en = corpus_en.split('\n')

# Realizamos el proceso anterior con el corpus español
for prefix in tqdm(nonBreaking_es):
    corpus_es = corpus_es.replace(prefix, prefix + '$$$')
corpus_es = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_es)
corpus_es = re.sub(r"\.\$\$\$", '', corpus_es)
corpus_es = re.sub(r"  +", " ", corpus_es)
corpus_es = corpus_es.split('\n')

100%|██████████| 38/38 [00:20<00:00,  1.85it/s]


In [9]:
target_vocab_size = (2**14) 

tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=target_vocab_size, )

tokenizer_es = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_es, target_vocab_size=target_vocab_size)

In [10]:
tokenizer_en

<SubwordTextEncoder vocab_size=16508>

In [11]:
vocab_size_en = tokenizer_en.vocab_size + 2  
vocab_size_es = tokenizer_es.vocab_size + 2 

vocab_size_en, vocab_size_en

(16510, 16510)

In [12]:
inputs = [[vocab_size_en -2] + tokenizer_en.encode(sentence) + [vocab_size_en -1] for sentence in corpus_en]

outputs = [[vocab_size_es -2] + tokenizer_es.encode(sentence) + [vocab_size_es -1] for sentence in corpus_es]

In [14]:
max_length = 22

idx_to_rm = [count for count, sentence in enumerate(inputs) if len(sentence) > max_length]

for idx in tqdm(reversed(idx_to_rm)):
    del inputs[idx]
    del outputs[idx]


idx_to_rm = [count for count, sentence in enumerate(outputs) if len(sentence) > max_length]

for idx in tqdm(reversed(idx_to_rm)):
    del inputs[idx]
    del outputs[idx]

len(inputs)

189128it [09:17, 339.10it/s]
209562it [04:23, 796.79it/s] 


1667596

In [15]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, 
    value=0, 
    padding="post", 
    maxlen=max_length
    )

outputs = tf.keras.preprocessing.sequence.pad_sequences(
    outputs, 
    value=0, 
    padding="post", 
    maxlen=max_length
    )

In [16]:
batch_size = 192
buffer_size = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()            # Cacheamos el dataset para agilizar el acceso a los datos en el entrenamiento
dataset = dataset.shuffle(buffer_size).batch(batch_size)   # Dividir el dataset en batches
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Ajustar caché para que el acceso a los bloques de datos sea más rápido

2023-01-10 18:41:40.713721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 18:41:40.743178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 18:41:40.743363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 18:41:40.744010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Construcción de modelo

## 1. Encoding posicional

tratamiento de los espacios vectoriales pares: $$PE_{pos, 2i} = sin(pos/10000^{2i / dmodel})$$
tratamiento de los espacios vectoriales impares: $$PE_{pos, 2i + 1} = cos(pos/10000^{2i / dmodel})$$

In [20]:
class PositionalEncoding(layers.Layer):

    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):  # pos : (seq_length, 1), i: (1, d_model)
        angles = 1 / np.power( 10000., (2*( i // 2 )) / np.float32(d_model) )
        return pos * angles  # (seq_length, d_model)

    def call(self, inputs):
        seq_length = inputs.shape.as_list()[-2] # max_length
        d_model = inputs.shape.as_list()[-1]

        angles = self.get_angles(
                            pos=np.arange(seq_length)[:, np.newaxis], 
                            i=np.arange(d_model)[np.newaxis, :],
                            d_model=d_model)

        angles[:, 0::2] = np.sin(angles[:, 0::2])  # aplicamos sin a las filas impares
        angles[:, 1::2] = np.cos(angles[:, 1::2])  # aplicamos cos a las filas impares

        pos_encoding = angles[np.newaxis, ...]   # Añadimos una dimensión extra al inicio para poder hacer el tratamiento de batches

        return inputs + tf.cast(pos_encoding, tf.float32)   # Combinamos las entradas con el encoding posicional
                                                            # que debe convertirse en tensor para poder ser utilizado

## 2. Mecanismo de atención
### 2.1 Cálculo de la atención

$$Attention(Q, K, V) = softmax \ \left( \dfrac{QK^{T}}{\sqrt{d_{k}}} \right) V$$


In [21]:
def scaled_dot_product_attention(queries, keys, values, mask):
    product = tf.matmul(queries, keys, transpose_b=True)

    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32)
    keys_dim_sqrt = tf.math.sqrt(keys_dim)

    scaled_product = product / keys_dim_sqrt

    if mask is not None:
        scaled_product += (mask * -1e9)

    softmax_scaled_product = tf.nn.softmax(scaled_product, axis=-1)

    attention = tf.matmul(softmax_scaled_product, values)

    return attention 

### 2.2 Creación del multihead attention

Utilizando la función anterior para el cálculo de la atención, crearemos una función para calcular paralelamente múltiples mecanismos de atención en los diferentes subespacios vectoriales en los que dividimos las entradas para aumentar la complejidad  

In [22]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, nb_projections):
        super(MultiHeadAttention, self).__init__()
        self.nb_projections = nb_projections

    def build(self, input_shape):
        # d_model es el tamaño de la dimensión de embedding
        self.d_model = input_shape[-1]
        
        # nos aseguramos de que el tamaño de la dimensión de embedding sea divisible 
        # entre el número de espacios vectoriales elegidos para el multiHeadAttention
        assert self.d_model % self.nb_projections == 0 

        # d_projections es el tamaño de cada una de las subdivisiones resultantes de 
        # dividir nuestra dimensión de embedding en subespacios vectoriales
        self.d_projections = self.d_model // self.nb_projections

        self.query_lin = layers.Dense(units=self.d_model)
        self.key_lin = layers.Dense(units=self.d_model)
        self.value_lin = layers.Dense(units=self.d_model)

        self.final_lin = layers.Dense(units=self.d_model)


    # Para dividir nuestra entrada en los diferentes espacios vectoriales 
    # para el multihead attention utilizaremos esta función
    #
    # forma inicial de antes de pasar por la función inputs: 
    #   - (batch_size, seq_length, d_model)

    def split_projetcions(self, inputs, batch_size):
        # shape es la variable que utilizaremos para definir la forma de las inputs después 
        # de ser divididas en subespacios vectoriales
        shape = (batch_size, -1, self.nb_projections, self.d_projections)

        # forma del input después del cambio: (batch_size, seq_length, nb_projections, d_projections)
        splitted_inputs = tf.reshape(inputs, shape=shape)

        # forma después de la permutación: (batch_size, nb_projections, seq_length, d_projections)
        permute_splitted_inputs = tf.transpose(splitted_inputs, perm=[0, 2, 1, 3])

        # el objeto retornado está dividido en batches con cada entrada de datos, cada entrada
        # de datos está dividido en los subespacios vectoriales, en cada subespacio encontraremos
        # la frase (sequence) y por último la porción de la dimensión de embedding del subespacio
        return permute_splitted_inputs


    # La función call contiene lo necesario para completar el mecanismo de atención, dónde procesamos 
    # individualmente Q K V en subespacios vectoriales con split_projections, después llamaremos a la
    # función que hará el cálculo de la atención, después sólo nos queda devolverle la forma que tenía
    # originalmente la entrada y enviarlo a la capa lineal final

    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        # llamamos a las capas lineales para cada Q K V
        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)


        # utilizamos la función para dividir nuestra entrada de datos en los subespacios vectoriales
        queries = self.split_projetcions(inputs=queries, batch_size=batch_size)
        keys = self.split_projetcions(inputs=keys, batch_size=batch_size)
        values = self.split_projetcions(inputs=values, batch_size=batch_size)


        # calculamos la attention en base al procesamiento individual de los subespacios de queries, 
        # keys y values, después le damos la forma necesaria para ser seguir el proceso, es decir:
        #  - (batch_size, nb_projections, seq_length, d_projections)

        attention = scaled_dot_product_attention(queries, keys, values, mask)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])


        # "concat_attention" es la variable que tiene todos los subespacios vectoriales generados para 
        # ser procesados en conjunto (formando el MultiHeadAttention) siendo enviados a la última capa 
        # lineal siendo su forma final:
        #  - (batch_size, seq_length, d_model)
        # es decir, la misma forma que tenía antes del proceso de división en subespacios

        concat_attention = tf.reshape(attention, shape=(batch_size, -1, self.d_model))
        outputs = self.final_lin(concat_attention)

        return outputs

## 3. Encoder
### 3.1 Creando la capa encoder

In [23]:
class encoderLayer(layers.Layer):

    def __init__(self, FC_units, nb_projections, dropout):
        super(encoderLayer, self).__init__()

        self.FC_units = FC_units
        self.nb_projections = nb_projections
        self.dropout = dropout
    
    def build(self, input_shape):
        self.d_model = input_shape[-1]

        self.multi_head_attention = MultiHeadAttention(self.nb_projections)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units=self.FC_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)


    def call(self, inputs, mask, training):
        attention = self.multi_head_attention(inputs, inputs, inputs, mask)
        attention = self.dropout_1(attention, training=training)
        attention = self.norm_1(attention + inputs)

        outputs = self.dense_1(attention) 
        outputs = self.dense_2(outputs) 
        outputs = self.dropout_2(outputs, training=training)
        outputs = self.norm_2(outputs + attention) 

        return outputs

### 3.2 Creamos el encoder utilizando la clase de capa

In [24]:
class Encoder(layers.Layer):

    def __init__(self, nb_layers, FC_units, nb_projections, dropout, vocab_size, d_model, name="encoder"):
        super(Encoder, self).__init__(name=name)
        
        self.nb_layers = nb_layers
        self.d_model = d_model

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)
        self.encoding_layers = [encoderLayer(FC_units=FC_units, nb_projections=nb_projections, dropout=dropout) for _ in range(nb_layers)]

    
    def call(self, inputs, mask, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)

        for i in range(self.nb_layers):
            outputs = self.encoding_layers[i](outputs, mask, training) 

        return outputs

## 4. Decoder
### 4.1 Clase decoder

In [25]:
class decoderLayer(layers.Layer):

    def __init__(self, FC_units, nb_projections, dropout):
        super(decoderLayer, self).__init__()

        self.FC_units = FC_units
        self.nb_projections = nb_projections
        self.dropout = dropout


    def build(self, input_shape):
        self.d_model = input_shape[-1]

        self.multi_head_attention_1 = MultiHeadAttention(self.nb_projections)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        self.multi_head_attention_2 = MultiHeadAttention(self.nb_projections)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units=self.FC_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_3 = layers.Dropout(rate=self.dropout)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)


    def call(self, inputs, encoder_outputs, mask_1, mask_2, training):
        attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1)
        attention = self.dropout_1(attention, training)
        attention = self.norm_1(attention + inputs)

        attention2 = self.multi_head_attention_2(attention, encoder_outputs, encoder_outputs, mask_2)
        attention2 = self.dropout_2(attention2, training)
        attention2 = self.norm_2(attention2 + inputs)

        outputs = self.dense_1(attention2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention)

        return outputs

### 4.2 Creamos el decoder

In [26]:
class Decoder(layers.Layer):

    def __init__(self, nb_layers, FC_units, nb_projections, dropout, vocab_size, d_model, name="decoder"):
        super(Decoder, self).__init__(name=name)

        self.d_model = d_model
        self.nb_layers = nb_layers

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        self.decoder_layers = [decoderLayer(FC_units, nb_projections, dropout) for _ in range(nb_layers)]


    def call(self, inputs, encoder_outputs, mask_1, mask_2, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.positional_encoding(outputs)
        outputs = self.dropout(outputs, training)

        for i in range(self.nb_layers):
            outputs = self.decoder_layers[i](outputs, encoder_outputs, mask_1, mask_2, training)

        return outputs

## Juntamos todo para crear el Transformer

In [27]:
class Transformer(tf.keras.Model):
    def __init__(self, vocab_size_encoder, vocab_size_decoder, d_model, nb_layers, FC_units, nb_projections, dropout, name="transformer"):
        super(Transformer, self).__init__(name=name)

        self.encoder = Encoder(nb_layers, FC_units, nb_projections, dropout, vocab_size_encoder, d_model)

        self.decoder = Decoder(nb_layers, FC_units, nb_projections, dropout, vocab_size_decoder, d_model)

        self.last_linear = layers.Dense(units=vocab_size_decoder, name="linear_output")


    def create_padding_mask(self, seq): # batch_size, seq_length
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]


    # Para que el modelo no pueda ver palabras futuras, debemos crear una matriz triangular       | 0 | 1 | 1 | 1 |
    # de manera que las palabras que queremos ocultar se vayan mostrando una por una según        | 0 | 0 | 1 | 1 |
    # el algoritmo vaya avanzando en la secuencia, de manera que en la primero sólo tendremos     | 0 | 0 | 0 | 1 |
    # acceso a la primera palabra y iremos desbloqueando las siguientes para evitar que el        | 0 | 0 | 0 | 0 |
    # transformer haga trampas y cree conexiones que fuera del entrenamiento resultarían ser 
    # inútiles. Para crear la matriz utilizaremos la sublibrería de tensorflow linalg (linear algebra) para tratar
    # los numeros que se encuentren en la diagonal superior y poder crear la matriz triangular. Las posiciones con 
    # un 0 serán visibles mientras que las que tengan un 1 tendrán máscara para no ser vistas por el modelo

    def create_look_ahead_mask(self, seq):
        seq_len = tf.shape(seq)[1]

        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask


    def call(self, encoder_inputs, decoder_inputs, training):
        encoder_mask = self.create_padding_mask(encoder_inputs)

        # decoder_mask_1                                                                            [8, 0, 35, 74, 0, 12]

        # combinamos las masks de cada secuencia con la look ahead mask para evitar que el        | 0 | 1 | 1 | 1 | 1 | 1 | 
        # modelo vea palabras futuras, de manera que con una secuencia de cómo la siguiente:      | 0 | 1 | 1 | 1 | 1 | 1 |
        # - [8, 0, 35, 74, 0, 12] nosotros obtendríamos la siguiente matriz en decoder:           | 0 | 1 | 0 | 1 | 1 | 1 |
        #                                                                                         | 0 | 1 | 0 | 0 | 1 | 1 |
        #                                                                                         | 0 | 1 | 0 | 0 | 1 | 1 |
        #                                                                                         | 0 | 1 | 0 | 0 | 1 | 0 |
        decoder_mask_1 = tf.maximum(
            self.create_padding_mask(decoder_inputs),
            self.create_look_ahead_mask(decoder_inputs) )

        decoder_mask_2 = self.create_padding_mask(encoder_inputs)

        encoder_outputs = self.encoder(encoder_inputs, encoder_mask, training)

        decoder_outputs = self.decoder(decoder_inputs, encoder_outputs, decoder_mask_1, decoder_mask_2, training)

        outputs = self.last_linear(decoder_outputs)

        return outputs

## Fase de entrenamiento

In [28]:
# Hiperparámetros

d_model = 512
nb_layers = 6
FC_units = 1024
nb_projections = 8
dropout_rate = 0.15

transformer = Transformer(vocab_size_encoder=vocab_size_en,
                          vocab_size_decoder=vocab_size_es,
                          d_model=d_model,
                          nb_layers=nb_layers, 
                          FC_units=FC_units,
                          nb_projections=nb_projections,
                          dropout=dropout_rate)

In [29]:
# Losses

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")

# From_logits=True, las salidas ya vienen en números salidos de una función softmax listos para calcular la loss
# Reduction="none", las pérdidas con se calculan haciendo la media del batch completo sinó cada muestra del batch individualmente 

# usaremos nuestra propia función de pérdida

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

$$learning\_ rate = d_{model}^{-0.5} \cdot min(step\_ num^{-0.5}, \ step\_ num \cdot warmup\_ steps^{-1.5})$$

In [30]:
# Learning rate schedule nos permite jugar con la variable step para variar el learning rate en función de step

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=3000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)  # rational sqrt, (step_num^{-0.5})
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [31]:
checkpoint_path = "./ckpt/max_l{}".format(max_length)

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [32]:
from IPython.display import clear_output # For Jupyter Notebook

epochs = 20 
lista_total = []
proggress = False
total = len(dataset)
count = 0
count_batch = 0
sections = 40

for epoch in range(epochs):
    if proggress:
        lista_total.append(proggress)
    lista_total.append("epoch: {}".format(epoch))

    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    for (batch, (encoder_inputs, targets)) in enumerate(dataset):
        
        decoder_inputs = targets[:, :-1]
        decoder_outputs_real = targets[:, 1:]

        with tf.GradientTape() as tape:
            predictions = transformer(encoder_inputs, decoder_inputs, True)
            loss = loss_function(decoder_outputs_real, predictions)
        
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables)) 

        train_loss(loss)
        train_accuracy(decoder_outputs_real, predictions)

        #if batch % 20 == 0:
        #    print("Epoch: {}. Lote: {}. Pérdidas: {:.4f}. Precisión {:.4f}".format(epoch+1, batch, train_loss.result(), train_accuracy.result()) ) 

        count_batch+=1
        
        if count_batch > (total / sections):
            clear_output(wait=True) # For Jupyter Notebook
            #os.system('clear')

            processed = total/sections * (count+1)
            proggress = "[Total: {} / {} batches / {} processed]\t[loss: {:.4f}, accuracy: {:.4f}]\t".format(int(total), int(processed), int(processed)*batch_size, 
                                                                                                             train_loss.result(), train_accuracy.result()
                                                                                                             ) + "\n" + "|"*(count+1)

            for line in lista_total:
                print(line)
            print(proggress)

            count+=1
            count_batch = 0

    ckpt_save_path = ckpt_manager.save()
    count = 0

    print("Guardando checkpoint para epoch {} en {}".format(epoch + 1, ckpt_save_path))
    print("Tiempo total de epoch {} sec".format(time.time() - start))

    count_batch = 0

epoch: 0
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 1.4516, accuracy: 0.2483]	
|||||||||||||||||||||||||||||||||||||||
epoch: 1
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.9979, accuracy: 0.2992]	
|||||||||||||||||||||||||||||||||||||||
epoch: 2
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.9179, accuracy: 0.3106]	
|||||||||||||||||||||||||||||||||||||||
epoch: 3
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.8737, accuracy: 0.3173]	
|||||||||||||||||||||||||||||||||||||||
epoch: 4
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.8430, accuracy: 0.3220]	
|||||||||||||||||||||||||||||||||||||||
epoch: 5
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.8203, accuracy: 0.3256]	
|||||||||||||||||||||||||||||||||||||||
epoch: 6
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.8024, accuracy: 0.3285]	
|||||||||||||||||||||||||||||||||||||||
epoch: 7
[Total: 8686 / 8468 batches / 1625856 processed]	[loss: 0.7882, acc

In [33]:
def evaluate(input_sentence):
    input_sentence = \
        [vocab_size_en-2] + tokenizer_en.encode(input_sentence) + [vocab_size_en-1]
    
    encoder_inputs = tf.expand_dims(input_sentence, axis=0)

    output = tf.expand_dims([vocab_size_es-2], axis=0)

    for _ in range(max_length):
        predictions = transformer(encoder_inputs, output, False)   # predictions shape: (1, seq_length, vocab_size_es)

        prediction = predictions[:, -1:, :]                         # nos quedamos con:
                                                                   #  - todas las secuencias (en éste caso sólo 1)
                                                                   #  - la última palabra usada del input
                                                                   #  - el vector con todas las palabras en español y la probabilidad de cada una

        # tomando el vector de palabras españolas, obtenemos el id de la que más probabilidad ha obtenido
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)   

        # la frase ha terminado, no hay nada más que decir
        if predicted_id == vocab_size_es-1:
            return tf.squeeze(output, axis=0)   # eliminamos la primera dimensión, la del batch, para quedarnos únicamente con la predicción

        output = tf.concat([output, predicted_id], axis=-1)

    # la frase ha terminado porque hemos alcanzado max_length
    return tf.squeeze(output, axis=0)


# la función evaluate nos devuelve los tokens de las palabras en castellano por lo que tenemos que traducirlas

def translate(sentence):
    # ya no es necesario usar tensores, por lo que ahora la trabajamos con numpy
    output = evaluate(sentence).numpy()

    predicted_sentence = tokenizer_es.decode(
        [i for i in output if i < vocab_size_es-2]
    )

    print("Entrada: {}".format(sentence))
    print("Salida: {}".format(predicted_sentence))

In [34]:
translate("This is a problem we have to solve.")

Entrada: This is a problem we have to solve.
Salida: Este es un problema que nos enfrentamos.


In [35]:
translate("This is a really powerful tool!")

Entrada: This is a really powerful tool!
Salida: Es un hecho que nos parece ser un hecho.
